In [0]:
import numpy 
import pandas 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.utils import np_utils 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold 
from sklearn.preprocessing import LabelEncoder 
from sklearn.pipeline import Pipeline
from sklearn import 
from keras import *

In [0]:
# fix random seed for reproducibility 
seed = 7 
numpy.random.seed(seed)

In [0]:
# load dataset 
dataframe = pandas.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv", header=None) 
dataset = dataframe.values 
X = dataset[:,0:4].astype(float) 
Y = dataset[:,4]

In [37]:
dataframe.head(3)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


The output variable contains three different string values. When modeling multiclass classiﬁcation problems using neural networks, it is good practice to reshape the output attribute from a vector that contains values for each class value to be a matrix with a boolean for each class value and whether or not a given instance has that class value or not. This is called one hot encoding or creating dummy variables from a categorical variable. For example, in this problem the three class values are Iris-setosa, Iris-versicolor and Iris-virginica. If we had the three observations:
- Iris-setosa 
- Iris-versicolor 
- Iris-virginica

We can do this by ﬁrst encoding the strings consistently to integers using the scikit-learn class LabelEncoder. Then convert the vector of integers to a one hot encoding using the Keras function to categorical().



In [0]:
# encode class values as integers 
encoder = LabelEncoder() 
encoder.fit(Y) 
encoded_Y = encoder.transform(Y) 
# convert integers to dummy variables (i.e. one hot encoded) 
dummy_y = np_utils.to_categorical(encoded_Y)

 ## Deﬁne The Neural Network Model
 4 inputs -> [4 hidden nodes] -> 3 outputs

Note that we use a sigmoid activation function in the output layer. This is to ensure the output values are in the range of 0 and 1 and may be used as predicted probabilities. Finally, the network uses the ecient ADAM gradient descent optimization algorithm with a logarithmic loss function, which is called categorical crossentropy in Keras.


In [52]:
# define baseline model 
def baseline_model(): 
  # create model 
  model = Sequential() 
  model.add(Dense(4, input_dim=4, init= 'normal' , activation= 'relu' )) 
  model.add(Dense(3, init= 'normal' , activation= 'sigmoid' )) 
  # Compile model 
  model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ]) 
  return model

model = Sequential() 
model.add(Dense(4, input_dim=4, init= 'normal' , activation= 'relu' )) 
model.add(Dense(3, init= 'normal' , activation= 'sigmoid' )) 
# Compile model 
model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ]) 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, input_dim=4, activation="relu", kernel_initializer="normal")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, activation="sigmoid", kernel_initializer="normal")`
  if sys.path[0] == '':


#### We can now create our KerasClassifier for use in scikit-learn. We can also pass arguments in the construction of the KerasClassifier class that will be passed on to the fit() function internally used to train the neural network. Here, we pass the number of epochs nb epoch as 200 and batch size as 5 to use when training the model. Debugging is also turned o↵ when training by setting verbose to 0.


## Evaluate The Model with k-Fold Cross Validation

We can now evaluate the neural network model on our training data. The scikit-learn library has excellent capability to evaluate models using a suite of techniques. The gold standard for evaluating machine learning models is k-fold cross validation. First we can deﬁne the model 66 evaluation procedure. Here, we set the number of folds to be 10 (an excellent default) and to shuffle the data before partitioning it.



In [0]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

Now we can evaluate our model (estimator) on our dataset (X and dummy y) using a 10-fold cross validation procedure (kfold). Evaluating the model only takes approximately 10 seconds and returns an object that describes the evaluation of the 10 constructed models for each of the splits of the dataset.


In [53]:
model.fit(X,dummy_y,nb_epoch=200, batch_size=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/200
150/150 [==============================] - 0s 650us/step - loss: 1.0958 - accuracy: 0.5867
Epoch 2/200
150/150 [==============================] - 0s 202us/step - loss: 1.0917 - accuracy: 0.6533
Epoch 3/200
150/150 [==============================] - 0s 211us/step - loss: 1.0857 - accuracy: 0.6667
Epoch 4/200
150/150 [==============================] - 0s 264us/step - loss: 1.0778 - accuracy: 0.6667
Epoch 5/200
150/150 [==============================] - 0s 230us/step - loss: 1.0666 - accuracy: 0.6667
Epoch 6/200
150/150 [==============================] - 0s 229us/step - loss: 1.0523 - accuracy: 0.6667
Epoch 7/200
150/150 [==============================] - 0s 233us/step - loss: 1.0351 - accuracy: 0.6667
Epoch 8/200
150/150 [==============================] - 0s 234us/step - loss: 1.0140 - accuracy: 0.6667
Epoch 9/200
150/150 [==============================] - 0s 269us/step - loss: 0.9900 - accuracy: 0.6667
Epoch 10/200
150/150 [==============================] - 0s 256us/step - l

In [55]:
scores = model.evaluate(X, dummy_y) 
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


150/150 [==============================] - 0s 124us/step
accuracy: 98.00%


In [62]:
from sklearn.model_selection import StratifiedKFold
# define 10-fold cross validation test harness 
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed) 
cvscores = [] 
for train, test in kfold.split(X, encoded_Y): 
  model=baseline_model()
  # Fit the model 
  model.fit(X[train], dummy_y[train], nb_epoch=150, batch_size=10, verbose=0) 
  # evaluate the model 
  scores = model.evaluate(X[test], dummy_y[test], verbose=0) 
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100)) 
  cvscores.append(scores[1] * 100)

print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores))) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, input_dim=4, activation="relu", kernel_initializer="normal")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, activation="sigmoid", kernel_initializer="normal")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


accuracy: 93.33%
accuracy: 93.33%
accuracy: 100.00%
accuracy: 93.33%
accuracy: 100.00%
accuracy: 93.33%
accuracy: 93.33%
accuracy: 66.67%
accuracy: 86.67%
accuracy: 100.00%
92.00% (+/- 9.33%)
